<a href="https://colab.research.google.com/github/GuendouziWassila/LSTMbidir_Binary_Text_Classif_spam/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.5/498.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

# Load data into Pandas DataFrame
columns = ["label", "message"]
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=columns)
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=columns)

# Display the first few rows of the training data
print("Training Data:")
print(train_data.head())

# Display the first few rows of the testing data
print("\nTesting Data:")
print(test_data.head())

--2024-01-01 15:27:41--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-01-01 15:27:41 (10.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-01-01 15:27:41--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
#------Preprocessing

#Convert Labels to Numerical Format:
label_encoder = LabelEncoder()

train_data['label'] = label_encoder.fit_transform(train_data['label'])
test_data['label'] = label_encoder.transform(test_data['label'])

# Tokenize the text : This step builds the vocabulary based on the training text data. It assigns a unique integer index to each word in the vocabulary.
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['message'])


# Convert text to sequences: The "texts_to_sequences method" converts each text message into a sequence of integers, where each integer represents a word's index in the vocabulary.
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
test_sequences = tokenizer.texts_to_sequences(test_data['message'])


# Pad sequences to a fixed length
max_length =250   #max(len(message.split()) for message in test_data['message']) # Choose an appropriate length
train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')
print (train_padded)


[[3666   36 2482 ...    0    0    0]
 [   3   29  281 ...    0    0    0]
 [  19    6  831 ...    0    0    0]
 ...
 [  47  464  337 ...    0    0    0]
 [  97   87 3665 ...    0    0    0]
 [  99   11  401 ...    0    0    0]]


In [ ]:
# Build the LSTM model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    keras.layers.Dense(32, activation='relu'),  # Add a dense layer
    keras.layers.GlobalAveragePooling1D(),  # Global average pooling to reduce sequence length to 1

    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_padded, train_data['label'], epochs=10, validation_data=(test_padded, test_data['label']))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_padded, test_data['label'])
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 12s 43ms/step - accuracy: 0.8763 - loss: 0.3882 - val_accuracy: 0.9691 - val_loss: 0.1119
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9853 - loss: 0.0618 - val_accuracy: 0.9806 - val_loss: 0.0596
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.9943 - loss: 0.0309 - val_accuracy: 0.9828 - val_loss: 0.0667
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9914 - loss: 0.0352 - val_accuracy: 0.9864 - val_loss: 0.0649
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.9976 - loss: 0.0120 - val_accuracy: 0.9842 - val_loss: 0.0507
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.9986 - loss: 0.0110 - val_accuracy: 0.9835 - val_loss: 0.0628
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.9991 - loss: 0.0073 - val_accuracy: 0.9885 - val_loss: 0.0636
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.9997 - loss: 0.0024 - val_a

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # Tokenize and pad the input text
  pred_sequence = tokenizer.texts_to_sequences([pred_text])
  pred_padded = tf.keras.preprocessing.sequence.pad_sequences(pred_sequence, maxlen=max_length, padding='post', truncating='post')

  # Make prediction using the trained model
  prediction= model.predict(pred_padded)

  # Convert the prediction to a probability value between 0 and 1
  probability = float(prediction[0][0])

  # Convert the probability to 'ham' or 'spam' based on a threshold (e.g., 0.5)
  label = 'spam' if probability >= 0.5 else 'ham'

  return [probability, label]

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
[0.0002464439894538373, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)

    print (prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[0.0002464439894538373, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[0.9996401071548462, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[0.00025331813958473504, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[1.0, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[1.0, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[0.00023936164507176727, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[0.0002656825818121433, 'ham']
You passed the challenge. Great job!
